In [2]:
import sys
sys.path.append('../')

In [3]:
import datetime
import math
from multiprocessing.pool import ThreadPool

import pandas
import numpy
import numpy.linalg
import scipy.linalg

from tqdm import tqdm

import lib.utils as utils
import lib.user_based_suggest as user_based_suggest

In [4]:
timus = pandas.read_csv('../data/timus.csv', index_col=0)

/home/ec2-user/mount_point/tfnewenv/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
timus.head()

,author_id,date,execution_time,judgement_result,language,memory_used,problem_id,submit_id,test_number
0,10004,2000-04-28 10:15:41,0.02,Accepted,Intel C++ 7,85,1000,184,0
1,10005,2000-05-12 13:13:38,0.00,Compilation error,Intel C++ 7,0,1000,185,0
2,10020,2000-09-22 15:58:11,0.01,Accepted,FreePascal 1.0.6,117,1000,220,0
3,10022,2000-09-26 17:17:34,0.01,Accepted,Intel C++ 7,81,1000,247,0
4,10033,2000-09-26 18:21:55,0.00,Compilation error,FreePascal 1.0.6,0,1000,276,0


In [6]:
filtered_timus = utils.filter_timus_by_unique_accepted(timus, lambda cnt: cnt > 400)

In [7]:
tc = utils.TestCase(filtered_timus, datetime.datetime.now() - datetime.timedelta(days=360))

In [8]:
sug = user_based_suggest.suggest(tc.get_train(), 5)

In [10]:
s = utils.get_first_ac_by_author(tc.get_test())

In [11]:
def get_score(suggest_by_author,
              next_accepted_by_author
             ):
    stoped_count = 0
    good_count = 0
    bad_count = 0
    for author in suggest_by_author:
        if author not in next_accepted_by_author:
            stoped_count += 1
            continue
        next_accepted = next_accepted_by_author[author]
        is_good = any([
            single_suggest.problem_index == next_accepted
            for single_suggest in suggest_by_author[author]
        ])
        if is_good:
            good_count += 1
        else:
            bad_count += 1
    print('good_count = {} bad_count = {} stoped_count = {}'.format(good_count, bad_count, stoped_count))
    return good_count / (good_count + bad_count)

In [12]:
get_score(sug, s)

good_count = 14 bad_count = 193 stoped_count = 165


0.06763285024154589

In [60]:
get_score(sug, s)

0.041666666666666664

In [108]:
U, S, VT = scipy.linalg.svd(solved_matrix, full_matrices=False)

In [109]:
SD = numpy.diag(S)

In [110]:
P = U.dot(SD).dot(VT)

In [111]:
numpy.linalg.norm(P - solved_matrix)

4.80038472230125e-12

In [119]:
P.shape

(18018, 1108)